In [1]:
from pathlib import Path

cwd = Path.cwd()


In [2]:
input_file = cwd / "input.txt"
input_simple = cwd / "input_simple.txt"


In [3]:
input = input_file.read_text().splitlines()
input_simple = input_simple.read_text().splitlines()

In [4]:
from collections import defaultdict
import numpy as np

def parse_input(input, step=2):
    infos = defaultdict(list)
    if step==2:
        seeds = np.array(input[0].split(": ")[-1].split(" ")).astype('int64')
        infos["seed"] = np.concatenate((np.arange(seeds[0],seeds[0]+seeds[1]),np.arange(seeds[2],seeds[2]+seeds[3]))).astype("int64")
        
    else:
        infos["seed"] = np.array(input[0].split(": ")[-1].split(" ")).astype("int64")
    line_number = {} 

    for idx, line in enumerate(input[1:]):
        if line == "seed-to-soil map:":
            line_number["seed-to-soil"] = idx + 1
        
        if line == "soil-to-fertilizer map:":
            line_number["soil-to-fertilizer"] = idx + 1
        
        if line == "fertilizer-to-water map:": 
            line_number["fertilizer-to-water"] = idx + 1
        
        if line == "water-to-light map:":
            line_number["water-to-light"] = idx + 1
        
        if line == "light-to-temperature map:":
            line_number["light-to-temperature"] = idx + 1
        
        if line == "temperature-to-humidity map:":
            line_number["temperature-to-humidity"] = idx + 1

        if line == "humidity-to-location map:":
            line_number["humidity-to-location"] = idx + 1
    

    for line in range(line_number["seed-to-soil"] +1, line_number["soil-to-fertilizer"]-1):
        infos["seed-to-soil"].append(np.array(input[line].split(" ")).astype("int64"))
    
    for line in range(line_number["soil-to-fertilizer"] +1, line_number["fertilizer-to-water"]-1):
        infos["soil-to-fertilizer"].append(np.array(input[line].split(" ")).astype("int64"))
    
    for line in range(line_number["fertilizer-to-water"] +1, line_number["water-to-light"]-1):
        infos["fertilizer-to-water"].append(np.array(input[line].split(" ")).astype("int64"))
    
    for line in range(line_number["water-to-light"] +1, line_number["light-to-temperature"]-1):
        infos["water-to-light"].append(np.array(input[line].split(" ")).astype("int64"))
     
    for line in range(line_number["light-to-temperature"] +1, line_number["temperature-to-humidity"]-1):
        infos["light-to-temperature"].append(np.array(input[line].split(" ")).astype("int64"))

    for line in range(line_number["temperature-to-humidity"] +1, line_number["humidity-to-location"]-1):
        infos["temperature-to-humidity"].append(np.array(input[line].split(" ")).astype("int64"))
    
    for line in range(line_number["humidity-to-location"] + 1, len(input)):
        infos["humidity-to-location"].append(np.array(input[line].split(" ")).astype("int64"))
    return infos

In [5]:
infos = parse_input(input, step=2)

In [116]:
def create_mapping(infos):

    result = defaultdict(list)

    for k in infos["seed"]:
        new_value = k.copy()
        
        for kinfo, v in infos.items():
            if kinfo != "seed":
                for kv in v:
                    if new_value>=kv[1] and new_value<(kv[1]+kv[2]):
                        dist = new_value - kv[1] 
                        new_value = kv[0] + dist
                        break
                
                result[k].append(new_value)
    
    return result


In [117]:
result = create_mapping(infos)

KeyboardInterrupt: 

In [6]:
def create_mapping_vectorise(infos):

    new_values = infos["seed"]
  
    for kinfo, v in infos.items():
        if kinfo != "seed":
            mapped = np.zeros_like(new_values)
            for kv in v:
                mask = (new_values>=kv[1]) & (new_values<(kv[1]+kv[2])) & (mapped==0)
                dist = new_values[mask] - kv[1] 
                new_values[mask] = kv[0] + dist
                mapped[mask] = 1
            
                
    return new_values

In [7]:
result = create_mapping_vectorise(infos)

In [ ]:
min_result= np.min([v[-1] for k, v in result.items()])
min_result

46